In [6]:
%pip install camelot-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 10.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 5.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2
Note: you may need to restart the kernel to use updated packages.


In [13]:
import camelot
import os
from tqdm import tqdm
import pandas as pd

def extract_tables_from_pdf(pdf_path, output_file, start_page, end_page):
    """
    Extracts tables from a PDF file using Camelot and saves them into an Excel file.

    Parameters:
        pdf_path (str): Path to the input PDF.
        output_file (str): Path to save the extracted tables Excel file.
        start_page (int): Starting page number.
        end_page (int): Ending page number.
    """
    
    all_tables = []
    output_file=f"/home/harikrishnan/Statathon/nco-semantic-search/data/processed/{output_file}"
    # Loop through the specified page range with a progress bar
    for page_num in tqdm(range(start_page, end_page + 1), desc="Extracting tables"):
        page = str(page_num)
        try:
            # First attempt: Lattice (for bordered tables)
            tables = camelot.read_pdf(f"/home/harikrishnan/Statathon/nco-semantic-search/data/raw/{pdf_path}", pages=page, flavor='lattice')

            # If no tables found, try Stream (for whitespace-separated tables)
            if tables.n == 0:
                tables = camelot.read_pdf(pdf_path, pages=page, flavor='stream')

            if tables.n == 0:
                print(f"[Page {page}] ❌ No tables found.")
            else:
                for table in tables:
                    df = table.df
                    df.columns = df.iloc[0]    # Use first row as header
                    df = df[1:]      
                    df.insert(0, "Page", page)  # Add a 'Page' column
                    all_tables.append(df)

        except Exception as e:
            print(f"[Page {page}] ⚠️ Error: {e}")

    # Combine and save tables if any were extracted
    if all_tables:
        combined_df = pd.concat(all_tables, ignore_index=True)
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        combined_df.to_csv(output_file, index=False)
        print(f"\n✅ All tables saved to: {output_file}")
        print("\n📌 Last few rows of the combined extracted table:")
        print(combined_df.tail())
    else:
        print("\n❌ No tables extracted from any page.")


In [14]:
extract_tables_from_pdf(
    pdf_path="nco_vol1.pdf",
    output_file="nco_vol1_alp_idx.csv",
    start_page=252,
    end_page=384
)


Extracting tables: 100%|██████████| 133/133 [01:41<00:00,  1.30it/s]

[Page 384] ⚠️ Error: [Errno 2] No such file or directory: 'nco_vol1.pdf'

✅ All tables saved to: /home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_vol1_alp_idx.csv

📌 Last few rows of the combined extracted table:
0    Page                               Occupational \nTitle NCO \n2015  \
3538  382  Working \nProprietor, \nStorage and \nWarehousing  1120.2700   
3539  382                  Working \nProprietor, \nTransport  1120.2300   
3540  382               Working \nProprietor, Water \nSupply  1120.0300   
3541  382  Working \nProprietor, Well \nDrilling/Propriet...  1120.1100   
3542  383                                 Zoologist, General  2131.0900   

0    NCO \n2004  
3538    1216.10  
3539    1215.10  
3540    1211.30  
3541    1213.30  
3542    2211.60  


In [15]:
import pandas as pd

# Path to the Excel file
excel_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_vol1_alp_idx.csv"

# Load Excel file
df = pd.read_csv(excel_path)

# Clean up columns with line breaks or extra spaces
col1 = df.columns[1]
col2 = df.columns[2]
col3 = df.columns[3]

# Normalize line breaks and whitespace in those 3 columns
df["_col1_clean"] = df[col1].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
df["_col2_clean"] = df[col2].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
df["_col3_clean"] = df[col3].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()

# Find duplicate header rows
mask = (
    (df["_col1_clean"] == "Occupational Title") &
    (df["_col2_clean"] == "NCO 2015") &
    (df["_col3_clean"] == "NCO 2004")
)

header_indices = df[mask].index

# Drop all but the first duplicate header row
if len(header_indices) > 1:
    df_cleaned = df.drop(header_indices[1:])
    print(f"✅ Removed {len(header_indices)-1} repeated header rows.")
else:
    df_cleaned = df
    print("ℹ️ No repeated headers found.")

# Drop helper columns
df_cleaned = df_cleaned.drop(columns=["_col1_clean", "_col2_clean", "_col3_clean"])

# 🔻 Drop the first column (by index)
df_cleaned = df_cleaned.iloc[:, 1:]

# Save back to Excel
df_cleaned.to_csv(excel_path, index=False)
print(f"📁 Cleaned Excel saved to: {excel_path}")


ℹ️ No repeated headers found.
📁 Cleaned Excel saved to: /home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_vol1_alp_idx.csv


In [99]:
import pdfplumber

# Set your PDF path
pdf_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/raw/nco_vol2a.pdf"
output_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/test.txt"

# Define page range (note: page indices are 0-based)
start_page = 13 - 1  # page 13
end_page = 14-1    # page 75

# Extract text from specified pages
extracted_text = []

with pdfplumber.open(pdf_path) as pdf:
    for i in range(start_page, end_page + 1):
        page = pdf.pages[i]

        # Page dimensions
        width = page.width
        height = page.height

        # Slightly offset from exact center to avoid text cut-off
        split_x = width * 0.52

        # Crop left and right columns
        left_col = page.crop((0, 0, split_x, height))
        right_col = page.crop((split_x, 0, width, height))

        # Extract text from each column
        left_text = left_col.extract_text()
        right_text = right_col.extract_text()

        # Combine left then right column text for that page
        combined_text = (left_text or "") + "\n" + (right_text or "")
        extracted_text.append(f"\n{combined_text}")

# # Save to text file
# with open(output_path, "w", encoding="utf-8") as f:
#     f.write("\n".join(extracted_text))

# print(f"✅ Extracted column-wise text saved to: {output_path}")
import re

# File paths
input_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_vol2a_p13_to_p75.txt"
output_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/test.txt"

# Set of exact lines to remove
EXACT_LINES_TO_REMOVE = {
    "National Classification of Occupations – 2015",
    "VOLUME",
    "VOLUME II A",
    "VOLUME II B"
}

# Regex patterns
division_pattern = re.compile(r"^Division\s+\d+$")
eii_pattern = re.compile(r"^E\s+II\s+A\s+\d+$")

# Clean storage
cleaned_lines = []
skip_next_if_division = False

# Process lines
for line in extracted_text:
    stripped = line.strip()

    if not stripped:
        continue

    # Remove exact matches
    if stripped in EXACT_LINES_TO_REMOVE:
        skip_next_if_division = True
        continue

    # Remove Division line if flagged
    if skip_next_if_division and division_pattern.match(stripped):
        skip_next_if_division = False
        continue

    # Remove E II A n lines
    if eii_pattern.match(stripped):
        continue

    # Reset flag if not matched
    skip_next_if_division = False

    # Keep the line
    cleaned_lines.append(stripped)

# Write cleaned output
with open(output_path, "w", encoding="utf-8") as f:
    f.write("\n".join(cleaned_lines))

print(f"✅ Cleaned file saved to:\n{output_path}")


✅ Cleaned file saved to:
/home/harikrishnan/Statathon/nco-semantic-search/data/processed/test.txt


In [118]:
import pdfplumber
from collections import defaultdict

pdf_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/raw/nco_vol2a.pdf"
start_page = 13 - 1  # zero-based index

def color_to_rgb_string(color):
    if isinstance(color, list) and len(color) == 3:
        return f"RGB({int(color[0]*255)}, {int(color[1]*255)}, {int(color[2]*255)})"
    return str(color)

with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[start_page]
    width = page.width
    height = page.height
    split_x = width * 0.52

    # Process both left and right columns
    for label, col in [("LEFT", page.crop((0, 0, split_x, height))), ("RIGHT", page.crop((split_x, 0, width, height)))]:
        print(f"\n=== {label} COLUMN ===")

        lines = defaultdict(list)

        # Group characters into lines by y-position
        for char in col.chars:
            y0 = round(char['top'], 1)  # small rounding to group close y-values
            lines[y0].append(char)

        # Sort lines by y-position
        for y0 in sorted(lines.keys()):
            chars = sorted(lines[y0], key=lambda c: c['x0'])  # left to right
            line_text = "".join(c['text'] for c in chars)

            if not line_text.strip():
                continue  # skip empty lines

            # Use the first character's font and color
            font = chars[0].get("fontname", "N/A")
            color = color_to_rgb_string(chars[0].get("non_stroking_color", "N/A"))

            print(f"[Font: {font} | Color: {color}] {line_text}")



=== LEFT COLUMN ===
[Font: ABCDEE+Calibri | Color: (0.0,)] National Classification of Occupations – 2015   
[Font: ABCDEE+Calibri,Bold | Color: (1.0,)] Division 
[Font: ABCDEE+Calibri,Bold | Color: (1.0,)] 1 
[Font: ABCDEE+Calibri,Bold | Color: (0.31, 0.506, 0.741)] Managers 
[Font: ABCDEE+Calibri | Color: (0.0,)] Managers plan, direct, co-ordinate and 
[Font: ABCDEE+Calibri | Color: (0.0,)] evaluate the overall activities of 
[Font: ABCDEE+Calibri | Color: (0.0,)] enterprises, governments and other 
[Font: ABCDEE+Calibri | Color: (0.0,)] organizations, or of organizational units 
[Font: ABCDEE+Calibri | Color: (0.0,)] within them, and formulate and review 
[Font: ABCDEE+Calibri | Color: (0.0,)] their policies, laws, rules and regulations. 
[Font: ABCDEE+Calibri | Color: (0.0,)] Tasks performed by managers usually 
[Font: ABCDEE+Calibri | Color: (0.0,)] include: formulating and advising on the 
[Font: ABCDEE+Calibri | Color: (0.0,)] policy, budgets, laws and regulations of 
[Font: ABC

In [6]:
import pdfplumber
from collections import defaultdict
import re

# === File Paths ===
pdf_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/raw/nco_vol2a.pdf"
output_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/test_quali.txt"

# === Page Range ===
start_page = 13 - 1
end_page = 585 - 1  # Inclusive

# === Font/Color Thresholds ===
TITLE_FONT_KEYWORD = "Bold"
TITLE_COLOR_RGB = (0.31, 0.506, 0.741)

# === Junk Filters ===
EXACT_LINES_TO_REMOVE = {
    "National Classification of Occupations – 2015",
    "VOLUME",
    "VOLUME II A",
    "VOLUME II B"
}
division_pattern = re.compile(r"^Division\s+\d+$")
eii_pattern = re.compile(r"^E\s+II\s+A\s+\d+$")
def remove_eii_patterns(text):
    # Matches "1 E II A", "2 E II A", ..., etc., and removes that pattern
    return re.sub(r"\b\d+\s+E\s+II\s+A\b", "", text)


# === Utilities ===
def is_same_color(c1, c2, tolerance=0.02):
    return all(abs(a - b) <= tolerance for a, b in zip(c1, c2))


def is_black(color):
    if isinstance(color, (int, float)):
        return color == 0.0
    return all(c == 0.0 for c in color)
def is_white(color, tolerance=0.01):
    if isinstance(color, (int, float)):
        return abs(color - 1.0) <= tolerance
    return all(abs(c - 1.0) <= tolerance for c in color)

def is_junk_line(text):
    stripped = text.strip()
    if not stripped:
        return False
    if stripped in EXACT_LINES_TO_REMOVE:
        return True
    if division_pattern.match(stripped):
        return True
    if eii_pattern.match(stripped):
        return True
    return False


def group_lines_by_y(chars):
    lines = defaultdict(list)
    for char in chars:
        y = round(char['top'], 1)
        lines[y].append(char)
    return lines


# === Main Extraction ===
title_desc_blocks = []
current_title = ""
current_desc = []
collecting_title = False
from tqdm import tqdm

with pdfplumber.open(pdf_path) as pdf:
    for page_num in tqdm(range(start_page, end_page + 1), desc="Processing pages"):
        page = pdf.pages[page_num]
        width, height = page.width, page.height
        split_x = width * 0.52

        for col in [page.crop((0, 0, split_x, height)), page.crop((split_x, 0, width, height))]:
            chars = col.chars
            lines = group_lines_by_y(chars)

            for y0 in sorted(lines.keys()):
                line_chars = sorted(lines[y0], key=lambda c: c['x0'])
                line_text = "".join(c['text'] for c in line_chars).strip()
                line_text = remove_eii_patterns(line_text).strip()

                if not line_text:
                    continue
                if is_junk_line(line_text):
                    continue

                font = line_chars[0].get("fontname", "")
                color = line_chars[0].get("non_stroking_color", [0, 0, 0])

                is_title = TITLE_FONT_KEYWORD in font and (
                    is_same_color(color, TITLE_COLOR_RGB) or is_black(color) or is_white(color)
                )

                if is_title:
                    if collecting_title:
                        current_title += " " + line_text
                    else:
                        if current_title and current_desc:
                            title_desc_blocks.append((current_title.strip(), " ".join(current_desc)))
                        current_title = line_text
                        current_desc = []
                        collecting_title = True
                else:
                    current_desc.append(line_text)
                    collecting_title = False

# === Final Block Save ===
if current_title and current_desc:
    title_desc_blocks.append((current_title.strip(), " ".join(current_desc)))

# === Write Output ===
with open(output_path, "w", encoding="utf-8") as f:
    for title, desc in title_desc_blocks:
        f.write(f"{title}\n{desc}\n\n")

print(f"✅ Extracted Title → Description pairs saved to:\n{output_path}")


Processing pages: 100%|██████████| 573/573 [00:37<00:00, 15.45it/s]

✅ Extracted Title → Description pairs saved to:
/home/harikrishnan/Statathon/nco-semantic-search/data/processed/test_quali.txt


# New extractor

In [8]:
import pdfplumber
from collections import defaultdict
import re
from tqdm import tqdm

# === Input files and page ranges ===
pdf_ranges = [
    ("/home/harikrishnan/Statathon/nco-semantic-search/data/raw/nco_vol2a.pdf", 13 - 1, 585 - 1),
    ("/home/harikrishnan/Statathon/nco-semantic-search/data/raw/nco_vol2b.pdf", 13 - 1, 579 - 1)
]

output_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/test_quali.txt"

# === Font/Color Thresholds ===
TITLE_FONT_KEYWORD = "Bold"
TITLE_COLOR_RGB = (0.31, 0.506, 0.741)

# === Junk Filters ===
EXACT_LINES_TO_REMOVE = {
    "National Classification of Occupations – 2015",
    "VOLUME",
    "VOLUME II A",
    "VOLUME II B"
}
division_pattern = re.compile(r"^Division\s+\d+$")
eii_pattern = re.compile(r"^E\s+II\s+A\s+\d+$")

def remove_eii_patterns(text):
    return re.sub(r"\b\d+\s+E\s+II\s+A\b", "", text)

# === Utilities ===
def is_same_color(c1, c2, tolerance=0.02):
    return all(abs(a - b) <= tolerance for a, b in zip(c1, c2))

def is_black(color):
    if isinstance(color, (int, float)):
        return color == 0.0
    return all(c == 0.0 for c in color)

def is_white(color, tolerance=0.01):
    if isinstance(color, (int, float)):
        return abs(color - 1.0) <= tolerance
    return all(abs(c - 1.0) <= tolerance for c in color)

def is_junk_line(text):
    stripped = text.strip()
    if not stripped:
        return False
    if stripped in EXACT_LINES_TO_REMOVE:
        return True
    if division_pattern.match(stripped):
        return True
    if eii_pattern.match(stripped):
        return True
    return False

def group_lines_by_y(chars):
    lines = defaultdict(list)
    for char in chars:
        y = round(char['top'], 1)
        lines[y].append(char)
    return lines

# === Main Extraction ===
title_desc_blocks = []
current_title = ""
current_desc = []
collecting_title = False

for pdf_path, start_page, end_page in pdf_ranges:
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in tqdm(range(start_page, end_page + 1), desc=f"Processing {pdf_path}"):
            page = pdf.pages[page_num]
            width, height = page.width, page.height
            split_x = width * 0.52

            for col in [page.crop((0, 0, split_x, height)),
                        page.crop((split_x, 0, width, height))]:
                chars = col.chars
                lines = group_lines_by_y(chars)

                for y0 in sorted(lines.keys()):
                    line_chars = sorted(lines[y0], key=lambda c: c['x0'])
                    line_text = "".join(c['text'] for c in line_chars).strip()
                    line_text = remove_eii_patterns(line_text).strip()

                    if not line_text or is_junk_line(line_text):
                        continue

                    font = line_chars[0].get("fontname", "")
                    color = line_chars[0].get("non_stroking_color", [0, 0, 0])

                    is_title = TITLE_FONT_KEYWORD in font and (
                        is_same_color(color, TITLE_COLOR_RGB) or
                        is_black(color) or is_white(color)
                    )

                    if is_title:
                        if collecting_title:
                            current_title += " " + line_text
                        else:
                            if current_title and current_desc:
                                title_desc_blocks.append((current_title.strip(), " ".join(current_desc)))
                            current_title = line_text
                            current_desc = []
                            collecting_title = True
                    else:
                        current_desc.append(line_text)
                        collecting_title = False

# Add last collected block
if current_title and current_desc:
    title_desc_blocks.append((current_title.strip(), " ".join(current_desc)))

# === Save Output ===
with open(output_path, "w", encoding="utf-8") as f:
    for title, desc in title_desc_blocks:
        f.write(f"{title}\n{desc}\n\n")

print(f"✅ Extracted Title → Description pairs saved to:\n{output_path}")


Processing /home/harikrishnan/Statathon/nco-semantic-search/data/raw/nco_vol2a.pdf: 100%|██████████| 573/573 [00:37<00:00, 15.39it/s]
Processing /home/harikrishnan/Statathon/nco-semantic-search/data/raw/nco_vol2b.pdf: 100%|██████████| 567/567 [00:35<00:00, 15.80it/s]

✅ Extracted Title → Description pairs saved to:
/home/harikrishnan/Statathon/nco-semantic-search/data/processed/test_quali.txt


In [9]:
import re
import pandas as pd

# Step 1: Load the text
with open('/home/harikrishnan/Statathon/nco-semantic-search/data/processed/test_quali.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Step 2: Regex pattern to capture the four fields
# Logic: Look for "Qualification Pack Details" before the code block
pattern = re.compile(
    r'Qualification Pack Details:\s*'
    r'QP NOS Reference\s+(?P<QP_Reference>\S+)\s+'
    r'QP NOS Name\s+(?P<QP_Name>.*?)\s+'
    r'NSQF Level\s+(?P<NSQF_Level>\d+).*?'
    r'(?P<Code>\d{4}\.\d{4})',   # Capture the code right after the details
    re.DOTALL
)

# Step 3: Extract all matches
records = []
for match in pattern.finditer(text):
    records.append({
        'Code': match.group('Code').strip(),
        'QP_NOS Reference': match.group('QP_Reference').strip(),
        'QP_NOS Name': match.group('QP_Name').strip(),
        'NSQF_Level': match.group('NSQF_Level').strip()
    })

# Step 4: Create DataFrame
df = pd.DataFrame(records)

# Step 5: Save or display
df.to_csv('/home/harikrishnan/Statathon/nco-semantic-search/data/processed/extracted_qp_codes.csv', index=False)
df


,Code,QP_NOS Reference,QP_NOS Name,NSQF_Level
0,1120.3402,MES/Q0207,Account Director (Advertising Agency),8
1,1120.3500,MES/Q0201,Sales Director (Media Org),8
2,1213.0102,ASC/Q6305,Quality Assurance Standards In Charge-Level 5,5
3,1213.0202,ASC/Q0603,Area Service Manager,6
4,1213.9900,ASC/Q0602,Territory Service Manager,5
...,...,...,...,...
709,9333.9900,LSC/Q1110,Loader/Unloader,2
710,9611.9900,TEL/Q2400,E Waste Collector,3
711,9623.0501,ASC/Q9601,PUC Attendant Level 2,2
712,9623.0601,ASC/Q9603,Tyre Inflation Attendant,2


In [13]:
import pandas as pd

# === File paths ===
qp_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/extracted_qp_codes.csv"
hierarchy_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_hierarchy_with_2004.csv"
output_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/hierarchy_with_qps.csv"

# === Load data ===
df_qp = pd.read_csv(qp_path)
df_hierarchy = pd.read_csv(hierarchy_path)

# === Merge so hierarchy columns come first ===
merged_df = df_hierarchy.merge(
    df_qp,
    left_on="Unit_Code",
    right_on="Code",
    how="left"
)

# === Save merged data ===
merged_df.to_csv(output_path, index=False)

print(f"✅ Merged dataset saved to:\n{output_path}")
merged_df.head()


✅ Merged dataset saved to:
/home/harikrishnan/Statathon/nco-semantic-search/data/processed/hierarchy_with_qps.csv


,Division,Division_Description,Sub_Division,Sub_Division_Description,Group,Group_Description,Family,Family_Description,Unit_Code,Unit_Title,Unit_Description,NCO_2004,Code,QP_NOS Reference,QP_NOS Name,NSQF_Level
0,1 Managers,"Managers plan, direct, co-ordinate and evaluat...","11 Chief Executives, Senior Officials and Legi...","Chief Executives, Senior Officials and Legisla...",111 Legislators and Senior Officials,"Legislators and senior officials determine, fo...",1111 Legislators,"Legislators determine, formulate, and direct p...",1111.01,"Elected Official, Union Government","Elected Official, Union Government serves in v...",1111.10,NaN,NaN,NaN,NaN
1,1 Managers,"Managers plan, direct, co-ordinate and evaluat...","11 Chief Executives, Senior Officials and Legi...","Chief Executives, Senior Officials and Legisla...",111 Legislators and Senior Officials,"Legislators and senior officials determine, fo...",1111 Legislators,"Legislators determine, formulate, and direct p...",1111.02,"Elected Official, State Government","Elected Official, State Government serves in v...",1112.10,NaN,NaN,NaN,NaN
2,1 Managers,"Managers plan, direct, co-ordinate and evaluat...","11 Chief Executives, Senior Officials and Legi...","Chief Executives, Senior Officials and Legisla...",111 Legislators and Senior Officials,"Legislators and senior officials determine, fo...",1111 Legislators,"Legislators determine, formulate, and direct p...",1111.03,"Elected Official, Local Bodies","Elected Official, Local Bodies serves in vario...",1113.10,NaN,NaN,NaN,NaN
3,1 Managers,"Managers plan, direct, co-ordinate and evaluat...","11 Chief Executives, Senior Officials and Legi...","Chief Executives, Senior Officials and Legisla...",111 Legislators and Senior Officials,"Legislators and senior officials determine, fo...",1111 Legislators,"Legislators determine, formulate, and direct p...",1111.99,"Legislators, Other","Elected Officials, Other include all other Ele...",1119.90,NaN,NaN,NaN,NaN
4,1 Managers,"Managers plan, direct, co-ordinate and evaluat...","11 Chief Executives, Senior Officials and Legi...","Chief Executives, Senior Officials and Legisla...",111 Legislators and Senior Officials,"Legislators and senior officials determine, fo...",1112 Senior Government Officials,Senior government officials advise governments...,1112.01,"Administrative Official, Union Government","Administrative Official, Union Government serv...",1121.10,NaN,NaN,NaN,NaN


In [24]:
import pandas as pd

# File path
merged_path = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/hierarchy_with_qps_clean.csv"


# Load merged data
df = pd.read_csv(merged_path)

# Select only the required columns


# Show 20 random rows
print(df[:].sample(20, random_state=42))


                                               Division  \
3133      8 Plant and Machine Operators, and Assemblers   
144                                          1 Managers   
1231          3 Technicians and Associate Professionals   
3000      8 Plant and Machine Operators, and Assemblers   
1052          3 Technicians and Associate Professionals   
2862      8 Plant and Machine Operators, and Assemblers   
410                                     2 Professionals   
2574                 7 Craft and Related Trades Workers   
315                                     2 Professionals   
2080                 7 Craft and Related Trades Workers   
1961                 7 Craft and Related Trades Workers   
3090      8 Plant and Machine Operators, and Assemblers   
2679      8 Plant and Machine Operators, and Assemblers   
1850                 7 Craft and Related Trades Workers   
2127                 7 Craft and Related Trades Workers   
2348                 7 Craft and Related Trades Workers 

In [27]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

texts = df['text'].tolist()
embeddings = model.encode(texts, show_progress_bar=True)

np.save('embeddings/nco_embeddings.npy', embeddings)


In [1]:
%pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 627.4 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [30]:
import re
from pathlib import Path

def clean_and_remove_annotations(input_path: str, output_path: str):
    remove_next_line = False
    cleaned_lines = []

    # === Patterns ===
    # Matches lines like: "7 E II A", "25 E II A"
    annotation_line_pattern = re.compile(r'^\s*\d+\s+(E\s+)?II\s+A\s*$', re.IGNORECASE)

    # Matches inline: "47 E II A", "63 II A", etc.
    embedded_annotation_pattern = re.compile(r'\b\d+\s+(E\s+)?II\s*A\b', re.IGNORECASE)

    # === Read all lines ===
    with open(input_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i]

        # Skip the line after "ISCO..." lines
        if remove_next_line:
            remove_next_line = False
            i += 1
            continue

        # Skip lines with "ISCO 08 Unit Group Details:"
        if "ISCO 08 Unit Group Details:" in line:
            remove_next_line = True
            i += 1
            continue

        # Skip full annotation lines like "63 II A", "7 E II A"
        if annotation_line_pattern.match(line.strip()):
            i += 1
            continue

        # Remove embedded annotations inside the line
        cleaned_line = embedded_annotation_pattern.sub('', line)

        # Clean up double/multiple spaces if it's not a blank line
        if cleaned_line.strip():
            cleaned_line = re.sub(r'\s{2,}', ' ', cleaned_line).strip()
            cleaned_lines.append(cleaned_line + '\n')
        else:
            # Preserve blank lines
            cleaned_lines.append('\n')

        i += 1

    # === Write cleaned output ===
    with open(output_path, 'w', encoding='utf-8') as f_out:
        f_out.writelines(cleaned_lines)

    print(f"✅ Fully cleaned text saved to: {output_path}")


if __name__ == "__main__":
    input_file_path = "../data/processed/test.txt"
    output_file_path = "../data/processed/cleaned_text2.txt"
    clean_and_remove_annotations(input_file_path, output_file_path)


✅ Fully cleaned text saved to: ../data/processed/cleaned_text2.txt


In [32]:
import re
import pandas as pd

def parse_isco_text(raw_text):
    """
    Parses raw ISCO occupational classification text into a structured DataFrame.
    Assumes each occupation entry starts with a code (e.g., '1111.0100') followed by a title,
    then a detailed multi-line description until the next code or end of text.
    """
    # Pattern to detect occupation code and title lines
    # Example code formats: '1111.0100', '1120', '11', etc.
    # This pattern captures codes like "1111.0100" or "1111"
    code_title_pattern = re.compile(r'^(\d{2,7}(?:\.\d{4})?)\s+(.+)$', re.MULTILINE)
    
    # Find all matches for codes and titles
    matches = list(code_title_pattern.finditer(raw_text))
    
    records = []
    for i, match in enumerate(matches):
        code = match.group(1).strip()
        title = match.group(2).strip()
        
        # Start of description: from this match end to next match start or end of text
        start_desc = match.end()
        end_desc = matches[i+1].start() if i+1 < len(matches) else len(raw_text)
        description = raw_text[start_desc:end_desc].strip()
        
        # Clean description: remove excessive newlines and spaces
        description = re.sub(r'\n+', ' ', description)
        description = re.sub(r'\s+', ' ', description)
        
        records.append({
            "Code": code,
            "Title": title,
            "Description": description
        })
    
    df = pd.DataFrame(records)
    return df

if __name__ == "__main__":
    # Read the raw text from a file, e.g. "isco_raw.txt"
    with open("../data/processed/cleaned_text2.txt", "r", encoding="utf-8") as f:
        raw_text = f.read()
    
    df_isco = parse_isco_text(raw_text)
    df_isco.to_csv("../data/processed/isco_dataset2.csv", index=False)
    print(f"Extracted {len(df_isco)} occupation records.")
    print(df_isco.head())


Extracted 1803 occupation records.
        Code                Title  \
0  6111.0100  Cultivator, General   
1  6111.0101         Paddy Farmer   
2  6111.0200     Cultivator, Crop   
3  6111.0201     Wheat Cultivator   
4  6111.0301     Maize Cultivator   

                                         Description  
0  Cultivators, General; Farmer, General; grows c...  
1  Paddy Farmer cultivates paddy as per the packa...  
2  Cultivator, Crop; Farmer, Crop grows field cro...  
3  Wheat Cultivator cultivates wheat as per the p...  
4  Maize Cultivator undertakes the cultivation of...  


In [34]:
import re
import pandas as pd

def parse_full_hierarchy(text):
    # Regex patterns for each level header
    division_pat = re.compile(r"^Division\s+(\d+)\s+(.*)$", re.IGNORECASE)
    sub_div_pat = re.compile(r"^Sub\s*Division\s+(\d+)\s+(.*)$", re.IGNORECASE)
    group_pat = re.compile(r"^Group\s+(\d+)\s+(.*)$", re.IGNORECASE)
    family_pat = re.compile(r"^Family\s+(\d+)\s+(.*)$", re.IGNORECASE)
    # Matches Unit lines: e.g., 1111.0200 Sewing Machine Operators
    unit_pat = re.compile(r"^(\d{4,7}(?:\.\d+)?)\s+(.*)$")

    # Store parsed records here
    records = []

    # Current state holders
    state = {
        "Division": None,
        "Division_Description": [],
        "Sub_Division": None,
        "Sub_Division_Description": [],
        "Group": None,
        "Group_Description": [],
        "Family": None,
        "Family_Description": [],
        "Unit_Code": None,
        "Unit_Title": None,
        "Unit_Description": []
    }

    def flush_unit():
        """Save the current unit record with all hierarchy."""
        if state["Unit_Code"] is not None:
            rec = {
                "Division": state["Division"],
                "Division_Description": " ".join(state["Division_Description"]).strip(),
                "Sub_Division": state["Sub_Division"],
                "Sub_Division_Description": " ".join(state["Sub_Division_Description"]).strip(),
                "Group": state["Group"],
                "Group_Description": " ".join(state["Group_Description"]).strip(),
                "Family": state["Family"],
                "Family_Description": " ".join(state["Family_Description"]).strip(),
                "Unit_Code": state["Unit_Code"],
                "Unit_Title": state["Unit_Title"],
                "Unit_Description": " ".join(state["Unit_Description"]).strip()
            }
            records.append(rec)
            # Reset unit info after flush
            state["Unit_Code"] = None
            state["Unit_Title"] = None
            state["Unit_Description"] = []

    def reset_lower_levels(level):
        """Reset all lower levels descriptions and identifiers after a new higher-level is found."""
        levels = ["Division", "Sub_Division", "Group", "Family", "Unit_Code"]
        level_indices = {lvl: idx for idx, lvl in enumerate(levels)}
        idx = level_indices[level]
        for lvl in levels[idx+1:]:
            state[lvl if not lvl.endswith("_Code") else "Unit_Code"] = None
            desc_key = lvl.replace("Code", "Description")
            if desc_key in state:
                state[desc_key] = []

    lines = text.splitlines()

    current_level = None  # Tracks current description target

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Match each header, starting with the highest level:
        div_m = division_pat.match(line)
        if div_m:
            flush_unit()

            reset_lower_levels("Division")

            state["Division"] = f"{div_m.group(1)} {div_m.group(2).strip()}"
            state["Division_Description"] = []
            current_level = "Division_Description"
            continue

        sub_div_m = sub_div_pat.match(line)
        if sub_div_m:
            flush_unit()

            reset_lower_levels("Sub_Division")

            state["Sub_Division"] = f"{sub_div_m.group(1)} {sub_div_m.group(2).strip()}"
            state["Sub_Division_Description"] = []
            current_level = "Sub_Division_Description"
            continue

        group_m = group_pat.match(line)
        if group_m:
            flush_unit()

            reset_lower_levels("Group")

            state["Group"] = f"{group_m.group(1)} {group_m.group(2).strip()}"
            state["Group_Description"] = []
            current_level = "Group_Description"
            continue

        family_m = family_pat.match(line)
        if family_m:
            flush_unit()

            reset_lower_levels("Family")

            state["Family"] = f"{family_m.group(1)} {family_m.group(2).strip()}"
            state["Family_Description"] = []
            current_level = "Family_Description"
            continue

        unit_m = unit_pat.match(line)
        if unit_m:
            flush_unit()

            reset_lower_levels("Unit_Code")

            state["Unit_Code"] = unit_m.group(1).strip()
            state["Unit_Title"] = unit_m.group(2).strip()
            state["Unit_Description"] = []
            current_level = "Unit_Description"
            continue

        # If line doesn't match headers, accumulate it in the current description
        if current_level:
            state[current_level].append(line)
        else:
            # If no current level, could log or skip
            pass

    # Flush the last unit at end of file
    flush_unit()

    df = pd.DataFrame(records)

    # Clean whitespace in all string columns
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.strip()

    return df

if __name__ == "__main__":
    # Replace this path with your file path
    input_file = "../data/processed/cleaned_text2.txt"

    with open(input_file, "r", encoding="utf-8") as f:
        raw_text = f.read()

    df_hierarchy = parse_full_hierarchy(raw_text)
    output_csv = "../data/processed/nco_full_hierarchy2.csv"
    df_hierarchy.to_csv(output_csv, index=False)

    print(f"Extracted {len(df_hierarchy)} records with detailed hierarchy.")
    print(df_hierarchy.head(10))


Extracted 1747 records with detailed hierarchy.
                                            Division  \
0  6 Skilled Agricultural, Forestry and Fishery W...   
1  6 Skilled Agricultural, Forestry and Fishery W...   
2  6 Skilled Agricultural, Forestry and Fishery W...   
3  6 Skilled Agricultural, Forestry and Fishery W...   
4  6 Skilled Agricultural, Forestry and Fishery W...   
5  6 Skilled Agricultural, Forestry and Fishery W...   
6  6 Skilled Agricultural, Forestry and Fishery W...   
7  6 Skilled Agricultural, Forestry and Fishery W...   
8  6 Skilled Agricultural, Forestry and Fishery W...   
9  6 Skilled Agricultural, Forestry and Fishery W...   

                                Division_Description  \
0  Skilled agricultural, forestry and fishery wor...   
1  Skilled agricultural, forestry and fishery wor...   
2  Skilled agricultural, forestry and fishery wor...   
3  Skilled agricultural, forestry and fishery wor...   
4  Skilled agricultural, forestry and fishery wor...   

In [43]:
import pandas as pd

# === File paths ===
file1 = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_full_hierarchy.csv"
file2 = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_full_hierarchy2.csv"
output_file = "/home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_merged_hierarchy.csv"

# === Load both CSVs ===
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# === Concatenate vertically ===
merged_df = pd.concat([df1, df2], ignore_index=True)

# === Optional: drop duplicates (if needed) ===
# merged_df.drop_duplicates(subset="Unit_Code", keep='first', inplace=True)

# === Save merged dataset ===
merged_df.to_csv(output_file, index=False)

print(f"✅ Files combined and saved to:\n{output_file}")


✅ Files combined and saved to:
/home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_merged_hierarchy.csv


In [25]:
import pandas as pd

df = pd.read_csv("/home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_hierarchy_with_2004.csv")

# Take the first 200 rows
df.head(200).to_csv("/home/harikrishnan/Statathon/nco-semantic-search/data/processed/sample_heirarchy.csv", index=False)


In [7]:
import pandas as pd

# Load files
df_idx = pd.read_csv('../data/processed/nco_vol1_alp_idx.csv', header=0)
df_idx.columns = ['Occupational_Title', 'NCO_2015', 'NCO_2004']

df_hier = pd.read_csv('../data/processed/nco_merged_hierarchy.csv')

# Normalize key columns (remove commas, strip)
df_idx['NCO_2015'] = df_idx['NCO_2015'].astype(str).str.replace(',', '', regex=False).str.strip()
df_idx['NCO_2004'] = df_idx['NCO_2004'].astype(str).str.replace(',', '', regex=False).str.strip()
df_hier['Unit_Code'] = df_hier['Unit_Code'].astype(str).str.replace(',', '', regex=False).str.strip()

# Create lookup dictionary
code_to_2004 = dict(zip(df_idx['NCO_2015'], df_idx['NCO_2004']))

# Test first 5 unit codes
print("🔍 Checking first 5 Unit_Code values:\n")

for unit_code in df_hier['Unit_Code'].head(5):
    match = code_to_2004.get(unit_code)
    if match:
        print(f"✅ Unit_Code: {unit_code} → NCO_2004: {match}")
    else:
        print(f"❌ Unit_Code: {unit_code} → NOT FOUND in index file")


✅ Done: NCO 2004 codes added where matched.


In [17]:
import pandas as pd

# === Load index file (title to NCO codes) ===
df_idx = pd.read_csv('../data/processed/nco_vol1_alp_idx.csv', header=0)
df_idx.columns = ['Occupational_Title', 'NCO_2015', 'NCO_2004']

# Clean index NCO_2015 and NCO_2004
df_idx['NCO_2015'] = df_idx['NCO_2015'].astype(str).str.replace(',', '', regex=False).str.strip()
df_idx['NCO_2004'] = df_idx['NCO_2004'].astype(str).str.replace(',', '', regex=False).str.strip()

# === Load hierarchy file ===
df_hier = pd.read_csv('../data/processed/nco_merged_hierarchy.csv')
df_hier['Unit_Code'] = df_hier['Unit_Code'].astype(str).str.replace(',', '', regex=False).str.strip()

# === Create a lookup dict from the index file ===
code_to_2004 = dict(zip(df_idx['NCO_2015'], df_idx['NCO_2004']))

# === Map NCO_2004 for each Unit_Code in the hierarchy file ===
df_hier['NCO_2004'] = df_hier['Unit_Code'].map(code_to_2004)

# === Save final enriched hierarchy ===
df_hier.to_csv('../data/processed/nco_hierarchy_with_2004.csv', index=False)

print("✅ NCO 2004 values added to hierarchy file based on matching Unit_Code with NCO 2015.")


✅ NCO 2004 values added to hierarchy file based on matching Unit_Code with NCO 2015.


In [24]:
import pandas as pd

# Load the CSV
df = pd.read_csv("/home/harikrishnan/Statathon/nco-semantic-search/data/processed/nco_hierarchy_with_2004.csv")

# Split 'Division' into numeric code and title
df[['Division_Code', 'Division_Title']] = df['Division'].str.extract(r'(\d+)\s+(.*)')

# Normalize
df['Division_Code'] = df['Division_Code'].astype(str).str.strip()
df['Division_Title'] = df['Division_Title'].str.strip().str.lower()

# Count rows where Division is '1 Managers'
count = df[(df['Division_Code'] == '4') ].shape[0]

print(f"✅ Number of rows for Division '1 Managers': {count}")


✅ Number of rows for Division '1 Managers': 118
